In [36]:
import pandas as pd
import numpy as np
import nltk
import string
import json
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datetime import datetime
import matplotlib.pyplot as plt

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import praw
import os

In [3]:
pip install --upgrade https://github.com/praw-dev/praw/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 26.6 MB 4.1 MB/s 0:00:05
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for praw: filename=praw-7.7.1.dev0-py3-none-any.whl size=190946 sha256=288d3749559fdf2ba9b159fe66a3cc7c5f98d6cf1833af7f24846d95f805a34d
  Stored in directory: /tmp/pip-ephem-wheel-cache-p8ayzg7b/wheels/ac/a7/39/b27772ff550949f2deaf392547263b8d1c5b6e0af53073b404
Successfully built praw


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
client_id = '_JsGDqLydGbjL9Ny3g_Q-A'
client_secret = '8uPqjwUKqxv2w3UbsVrOOh53lZcCeg'
user_agent = 'CIS509'

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

In [9]:
def get_top_posts(subreddit, start_date, end_date, limit=100):
    posts = []
    for post in reddit.subreddit(subreddit).top('all', limit=limit):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if start_date <= post_date <= end_date:
            posts.append(post)
    return posts

start_date = datetime(2021, 1, 1)
end_date = datetime(2023, 12, 31)
subreddit_name = 'Nike'

top_posts = get_top_posts(subreddit_name, start_date, end_date)


<ipython-input-9-53e0119ac18d>:3: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for post in reddit.subreddit(subreddit).top('all', limit=limit):
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [10]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    text = ' '.join([word for word in word_tokens if word not in stop_words])
    return text

sia = SentimentIntensityAnalyzer()

In [11]:
def categorize_sentiment(score):
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

In [12]:
def process_post(post):
    post_info = {
        'id': post.id,
        'title': post.title,
        'url': post.url,
        'score': post.score,
        'comments_count': post.num_comments,
        'created_utc': post.created_utc
    }

    post_info['preprocessed_title'] = preprocess_text(post.title)
    sentiment_scores = sia.polarity_scores(post_info['preprocessed_title'])
    post_info['sentiment_scores'] = sentiment_scores
    post_info['sentiment'] = categorize_sentiment(sentiment_scores['compound'])

    comments = []
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        if not comment.body == '[deleted]' and not comment.body == '[removed]':
            comment_info = {
                'id': comment.id,
                'body': comment.body,
                'score': comment.score,
                'created_utc': comment.created_utc
            }
            comment_info['preprocessed_body'] = preprocess_text(comment.body)
            sentiment_scores = sia.polarity_scores(comment_info['preprocessed_body'])
            comment_info['sentiment_scores'] = sentiment_scores
            comment_info['sentiment'] = categorize_sentiment(sentiment_scores['compound'])
            comments.append(comment_info)

    post_info['comments'] = comments

    return post_info

processed_data = [process_post(post) for post in top_posts]

json_filename = 'nike_top_posts_sentiment_vader.json'
with open(json_filename, 'w') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [13]:
print(f"\nSentiment analysis is complete. The results are saved in {json_filename}.")



Sentiment analysis is complete. The results are saved in nike_top_posts_sentiment_vader.json.


In [14]:
def display_results(processed_data):
    for post_info in processed_data:
        print(f"Post ID: {post_info['id']}")
        print(f"Title: {post_info['title']}")
        print(f"Sentiment: {post_info['sentiment'].capitalize()}")
        print(f"Sentiment Scores: {post_info['sentiment_scores']}")
        print("\nTop comments:")
        for comment_info in post_info['comments'][:5]:
            print(f"\tComment ID: {comment_info['id']}")
            print(f"\tBody: {comment_info['body'][:100]}...")
            print(f"\tSentiment: {comment_info['sentiment'].capitalize()}")
            print(f"\tSentiment Scores: {comment_info['sentiment_scores']}\n")
        print("------------------------------------------------------\n")

display_results(processed_data)


Post ID: lan856
Title: Yea I was that mom that messaged Nike asking questions 😅. Thought it wouldn't hurt to spam the internet to get Nike shoes to look at accommodating kids with special needs. To date we've yet to find a shoe that actually fits with Maddy's AFOs.
Sentiment: Positive
Sentiment Scores: {'neg': 0.0, 'neu': 0.767, 'pos': 0.233, 'compound': 0.764}

Top comments:
	Comment ID: glpkixc
	Body: This would be awesome to see - come on Nike - just do it!...
	Sentiment: Positive
	Sentiment Scores: {'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6249}

	Comment ID: glpl8xn
	Body: Good luck hopefully they can pull something off but it might have been just too hard to make that de...
	Sentiment: Positive
	Sentiment Scores: {'neg': 0.07, 'neu': 0.5, 'pos': 0.43, 'compound': 0.802}

	Comment ID: glp1vkf
	Body: Go to r/sneakers as well...
	Sentiment: Positive
	Sentiment Scores: {'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.2732}

	Comment ID: glr67b7
	Body: Could we also

In [15]:
def count_sentiments(processed_data):
    post_sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}
    comment_sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}

    for post_info in processed_data:
        post_sentiment_counts[post_info['sentiment']] += 1

        for comment_info in post_info['comments']:
            comment_sentiment_counts[comment_info['sentiment']] += 1

    return post_sentiment_counts, comment_sentiment_counts

post_sentiment_counts, comment_sentiment_counts = count_sentiments(processed_data)

print("Post sentiment counts:")
for sentiment, count in post_sentiment_counts.items():
    print(f"{sentiment.capitalize()}: {count}")

print("\nComment sentiment counts:")
for sentiment, count in comment_sentiment_counts.items():
    print(f"{sentiment.capitalize()}: {count}")


Post sentiment counts:
Positive: 14
Negative: 5
Neutral: 22

Comment sentiment counts:
Positive: 826
Negative: 314
Neutral: 671


In [16]:
import csv



In [17]:
def save_sentiments_to_csv(category_sentiments, file_path):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Category', 'Positive', 'Negative', 'Neutral'])
        for category, sentiments in category_sentiments.items():
            row = [category, sentiments['positive'], sentiments['negative'], sentiments['neutral']]
            writer.writerow(row)

In [18]:
def group_sentiments_by_categories(processed_data):
    categories = {
        'Nike Shoes': ['shoe', 'shoes', 'sneaker', 'sneakers', 'air force 1', 'jordan 1', 'air max', 'air max 90', 'air max 95', 'air max 97', 'air max 270', 'air vapormax', 'cortez', 'presto', 'react', 'pegasus'],
        'Nike Sportswear': ['sportswear', 'clothing', 'apparel', 'gear', 'jersey', 'pants', 'hoodie', 'jacket', 'shorts', 't-shirt', 'socks', 'compression', 'leggings', 'sports bra', 'windbreaker', 'fleece', 'sweatshirt', 'sweatpants'],
        'Nike Hypebeast': ['jordan', 'hypebeast', 'dunk', 'air max', 'collab', 'collaboration', 'off-white', 'travis scott', 'supreme', 'sacai', 'undefeated', 'cactus plant flea market', 'ambush', 'comme des garcons', 'fear of god', 'stussy', 'union']
    }


    category_sentiments = {category: {'positive': 0, 'negative': 0, 'neutral': 0} for category in categories}

    for post_info in processed_data:
        post_sentiment = post_info['sentiment']
        post_title = post_info['title'].lower()

        for category, keywords in categories.items():
            if any(keyword in post_title for keyword in keywords):
                category_sentiments[category][post_sentiment] += 1

                for comment_info in post_info['comments']:
                    comment_sentiment = comment_info['sentiment']
                    category_sentiments[category][comment_sentiment] += 1

    return category_sentiments



In [19]:
category_sentiments = group_sentiments_by_categories(processed_data)

file_path = '/content/drive/MyDrive/category_sentiments.csv'
save_sentiments_to_csv(category_sentiments, file_path)


In [20]:
import csv

def save_processed_data_to_csv(processed_data, folder_path):
    csv_filename = os.path.join(folder_path, 'processed_data.csv')
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['post_id', 'title', 'url', 'score', 'comments_count', 'created_utc', 'sentiment'])
        
        for post_info in processed_data:
            csv_writer.writerow([post_info['id'], post_info['title'], post_info['url'], post_info['score'],
                                 post_info['comments_count'], post_info['created_utc'], post_info['sentiment']])
                                 
    print(f"\nProcessed data is saved as a CSV file in {csv_filename}.")

# Save processed_data as a CSV file
folder_path = '/content/drive/MyDrive'
save_processed_data_to_csv(processed_data, folder_path)



Processed data is saved as a CSV file in /content/drive/MyDrive/processed_data.csv.


In [21]:
!pip install -q gensim

In [22]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

def perform_topic_modeling(processed_data, num_topics=5, num_words=10):
    # Prepare the corpus for topic modeling
    texts = [post_info['preprocessed_title'].split() for post_info in processed_data]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Train the LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)

    # Print the topics
    topics = lda_model.print_topics(num_words=num_words)
    print("\nTopics found via LDA:")
    for topic in topics:
        print(f"Topic {topic[0]}: {topic[1]}")
    
    return lda_model, dictionary, corpus

lda_model, dictionary, corpus = perform_topic_modeling(processed_data)



Topics found via LDA:
Topic 0: 0.043*"jordan" + 0.023*"air" + 0.023*"1" + 0.023*"fake" + 0.012*"bought" + 0.012*"got" + 0.012*"create" + 0.012*"im" + 0.012*"swoosh" + 0.012*"nike"
Topic 1: 0.031*"fun" + 0.021*"guys" + 0.021*"made" + 0.012*"’" + 0.012*"app" + 0.012*"nike" + 0.012*"im" + 0.012*"custom" + 0.012*"let" + 0.012*"share"
Topic 2: 0.047*"nike" + 0.035*"’" + 0.018*"pair" + 0.018*"shoes" + 0.013*"thought" + 0.013*"working" + 0.013*"home" + 0.013*"guy" + 0.013*"work" + 0.013*"first"
Topic 3: 0.026*"fake" + 0.026*"nikes" + 0.018*"first" + 0.018*"vendor" + 0.018*"day" + 0.018*"tokyo" + 0.018*"march" + 0.018*"shinjuku" + 0.010*"collection" + 0.010*"’"
Topic 4: 0.017*"shoe" + 0.017*"id" + 0.017*"violent" + 0.017*"ig" + 0.017*"meme" + 0.017*"upload" + 0.017*"saw" + 0.017*"video" + 0.017*"here…" + 0.017*"sorry"


In [23]:
import csv

def save_comments_sentiments_to_csv(processed_data, folder_path):
    csv_filename = os.path.join(folder_path, 'comments_sentiments.csv')
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['post_id', 'comment_id', 'body', 'score', 'created_utc', 'sentiment'])
        
        for post_info in processed_data:
            post_id = post_info['id']
            for comment_info in post_info['comments']:
                csv_writer.writerow([post_id, comment_info['id'], comment_info['body'], comment_info['score'],
                                     comment_info['created_utc'], comment_info['sentiment']])
                                     
    print(f"\nComments sentiment data is saved as a CSV file in {csv_filename}.")

# Save comments_sentiments as a CSV file
folder_path = '/content/drive/MyDrive'
save_comments_sentiments_to_csv(processed_data, folder_path)



Comments sentiment data is saved as a CSV file in /content/drive/MyDrive/comments_sentiments.csv.


In [24]:
!pip install -q pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
google-colab 1.0.0 requires pandas~=1.5.3, but you have pandas 2.0.1 which is incompatible.


In [25]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

def visualize_topics(lda_model, corpus, dictionary):
    vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis_data)

# Display topics as a graph
visualize_topics(lda_model, corpus, dictionary)


ERROR:concurrent.futures:exception calling callback for <Future at 0x7feca2564610 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.9/dist-packages/joblib/parallel.py", line 385, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.9/dist-packages/joblib/parall

BrokenProcessPool: ignored

In [26]:
def summary_statistics(processed_data):
    total_posts = len(processed_data)
    total_score = 0
    total_comments = 0

    for post_info in processed_data:
        total_score += post_info['score']
        total_comments += post_info['comments_count']

    avg_score = total_score / total_posts
    avg_comments = total_comments / total_posts

    print(f"Total posts: {total_posts}")
    print(f"Average score: {avg_score:.2f}")
    print(f"Average number of comments: {avg_comments:.2f}")

# Call the summary_statistics function with the processed_data
summary_statistics(processed_data)


Total posts: 41
Average score: 479.34
Average number of comments: 44.90


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
from collections import Counter

def term_statistics(processed_data):
    title_terms = []
    comment_terms = []
    authors = set()
    total_submissions = 0
    total_comments = 0
    total_comment_replies = 0
    total_score = 0
    total_num_comments = 0
    comment_word_count = []
    comment_lengths = []

    for post_info in processed_data:
        total_submissions += 1
        authors.add(post_info.get('author', 'unknown'))
        title_terms.extend(post_info['preprocessed_title'].split())
        total_score += post_info['score']
        total_num_comments += post_info['comments_count']

        for comment_info in post_info['comments']:
            total_comments += 1
            comment_terms.extend(comment_info['preprocessed_body'].split())
            authors.add(comment_info.get('author', 'unknown'))
            comment_word_count.append(len(comment_info['preprocessed_body'].split()))
            comment_lengths.append(len(comment_info['body']))
            if comment_info.get('parent_id') and comment_info['parent_id'].startswith('t1_'):
                total_comment_replies += 1

    title_term_counts = Counter(title_terms)
    comment_term_counts = Counter(comment_terms)

    print("Summary statistics:")
    print(f"Number of posts: {total_submissions}")
    print(f"Number of comments: {total_comments}")
    print(f"Average score: {total_score / total_submissions:.2f}")
    print(f"Average number of comments: {total_num_comments / total_submissions:.2f}")
    print(f"Total number of unique words: {len(title_term_counts) + len(comment_term_counts)}")
    print(f"Total number of words in the corpus: {sum(title_term_counts.values()) + sum(comment_term_counts.values())}")
    print(f"Average number of words in comments: {sum(comment_word_count) / len(comment_word_count):.2f}")
    print(f"Minimum number of words in comments (without stop words): {min(comment_word_count)}")
    print(f"Maximum number of words in comments (without stop words): {max(comment_word_count)}")
    print(f"Number of unique authors: {len(authors)}")
    print(f"Number of comments replying to other comments: {total_comment_replies}")
    print(f"Total number of submissions: {total_submissions}")

    print("\nTerm statistics for post titles:")
    for term, count in title_term_counts.most_common(10):
        print(f"{term}: {count}")

    print("\nTerm statistics for comments:")
    for term, count in comment_term_counts.most_common(10):
        print(f"{term}: {count}")

# Call the term_statistics function with the processed_data
term_statistics(processed_data)


Summary statistics:
Number of posts: 41
Number of comments: 1811
Average score: 479.34
Average number of comments: 44.90
Total number of unique words: 4305
Total number of words in the corpus: 15510
Average number of words in comments: 8.39
Minimum number of words in comments (without stop words): 0
Maximum number of words in comments (without stop words): 263
Number of unique authors: 1
Number of comments replying to other comments: 0
Total number of submissions: 41

Term statistics for post titles:
nike: 11
’: 8
pair: 5
first: 5
fake: 5
thought: 4
nikes: 4
jordan: 4
shoes: 3
custom: 3

Term statistics for comments:
’: 543
nike: 165
shoes: 142
like: 136
get: 116
would: 98
pair: 90
good: 86
got: 84
look: 81


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
import numpy as np
from collections import defaultdict
from datetime import datetime

def sentiment_statistics(processed_data):
    sentiment_categories = ['positive', 'neutral', 'negative']
    post_stats = {sentiment: {'count': 0, 'scores': [], 'highest_score_posts': [], 'lowest_score_posts': [], 'year_count': defaultdict(int)} for sentiment in sentiment_categories}
    comment_stats = {sentiment: {'count': 0, 'scores': [], 'highest_score_comments': [], 'lowest_score_comments': [], 'year_count': defaultdict(int)} for sentiment in sentiment_categories}

    for post_info in processed_data:
        post_sentiment = post_info['sentiment']
        post_stats[post_sentiment]['count'] += 1
        post_stats[post_sentiment]['scores'].append(post_info['score'])
        post_stats[post_sentiment]['highest_score_posts'].append(post_info)
        post_stats[post_sentiment]['lowest_score_posts'].append(post_info)
        post_stats[post_sentiment]['year_count'][datetime.utcfromtimestamp(post_info['created_utc']).year] += 1

        for comment_info in post_info['comments']:
            comment_sentiment = comment_info['sentiment']
            comment_stats[comment_sentiment]['count'] += 1
            comment_stats[comment_sentiment]['scores'].append(comment_info['score'])
            comment_stats[comment_sentiment]['highest_score_comments'].append(comment_info)
            comment_stats[comment_sentiment]['lowest_score_comments'].append(comment_info)
            comment_stats[comment_sentiment]['year_count'][datetime.utcfromtimestamp(comment_info['created_utc']).year] += 1

    for sentiment in sentiment_categories:
        post_stats[sentiment]['highest_score_posts'] = sorted(post_stats[sentiment]['highest_score_posts'], key=lambda x: x['score'], reverse=True)[:3]
        post_stats[sentiment]['lowest_score_posts'] = sorted(post_stats[sentiment]['lowest_score_posts'], key=lambda x: x['score'])[:3]
        comment_stats[sentiment]['highest_score_comments'] = sorted(comment_stats[sentiment]['highest_score_comments'], key=lambda x: x['score'], reverse=True)[:3]
        comment_stats[sentiment]['lowest_score_comments'] = sorted(comment_stats[sentiment]['lowest_score_comments'], key=lambda x: x['score'])[:3]

    def print_sentiment_stats(stats, name, stat_name_highest, stat_name_lowest):
        print(f"\n{name} sentiment breakdown:")
        print(f"{'Sentiment':<10} {'Count':<10} {'Percentage':<12} {'Total Score':<12} {'Avg Score':<12} {'Std Dev':<12} {'Median':<12} {'Top 3 Highest Score'} {'Top 3 Lowest Score'} {'Yearly Breakdown'}")
        total_count = sum([s['count'] for s in stats.values()])
        for sentiment, data in stats.items():
            percentage = data['count'] / total_count * 100
            avg_score = np.mean(data['scores']) if data['count'] > 0 else 0
            std_dev = np.std(data['scores']) if data['count'] > 0 else 0
            median = np.median(data['scores']) if data['count'] > 0 else 0
            top_3_highest_ids = ", ".join([str(post['id']) for post in data[stat_name_highest]])
            top_3_lowest_ids = ", ".join([str(post['id']) for post in data[stat_name_lowest]])
            yearly_breakdown = ", ".join([f"{year}: {count}" for year, count in data['year_count'].items()])

            print(f"{sentiment.capitalize():<10} {data['count']:<10} {percentage:<12.2f}% {sum(data['scores']):<12} {avg_score:<12.2f} {std_dev:<12.2f} {median:<12.2f} {top_3_highest_ids} {top_3_lowest_ids} {yearly_breakdown}")

    print_sentiment_stats(post_stats, "Post", 'highest_score_posts', 'lowest_score_posts')
    print_sentiment_stats(comment_stats, "Comment", 'highest_score_comments', 'lowest_score_comments')

# Call the sentiment_statistics function with the processed_data
sentiment_statistics(processed_data)



Post sentiment breakdown:
Sentiment  Count      Percentage   Total Score  Avg Score    Std Dev      Median       Top 3 Highest Score Top 3 Lowest Score Yearly Breakdown
Positive   14         34.15       % 7730         552.14       324.60       352.50       lan856, kqlxpi, ksf5dy mp3io2, qhf0ta, tgxy8g 2021: 7, 2022: 4, 2023: 3
Neutral    22         53.66       % 9779         444.50       168.97       379.00       lngp1d, qw79q6, w9cr5e 12o87sr, mi0jxa, sr98h1 2021: 16, 2022: 4, 2023: 2
Negative   5          12.20       % 2144         428.80       94.97        375.00       uwzo1h, y5rdnq, x32dw7 11kzoy6, 11ri3wi, x32dw7 2022: 3, 2023: 2

Comment sentiment breakdown:
Sentiment  Count      Percentage   Total Score  Avg Score    Std Dev      Median       Top 3 Highest Score Top 3 Lowest Score Yearly Breakdown
Positive   826        45.61       % 3894         4.71         12.70        2.00         j8fg7kv, j8fhd3q, go0np8v j2h35d4, gkhds97, gpfqhj9 2021: 336, 2022: 230, 2023: 260
Neutral   

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
